In [1]:
# Block 1: Importing Libraries
import tarfile
import librosa
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam


In [2]:
# Block 2: Extracting Data
# Extract train_mp3s.tar
with tarfile.open('train_mp3s.tar', 'r') as tar:
    tar.extractall('train_mp3s')

# Extract test_mp3s.tar
with tarfile.open('test_mp3s.tar', 'r') as tar:
    tar.extractall('test_mp3s')

train_labels = np.loadtxt('train_label.txt', dtype=int)


In [3]:

# Block 3: Preprocessing Functions
import concurrent.futures
import librosa
import numpy as np
import os

def preprocess_audio(file_path):
    try:
        print(f"Attempting to load file: {file_path}")
        audio, sample_rate = librosa.load(file_path, sr=None, res_type='kaiser_fast')
        print(f"File loaded successfully: {file_path}")
        
        # Extracting multiple features
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
        contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        
        features = np.concatenate((np.mean(mfccs, axis=1), np.mean(chroma, axis=1), 
                                   np.mean(mel, axis=1), np.mean(contrast, axis=1)))
        print(f"Features extracted for {file_path}: {features.shape}")
        
        return features
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None

def prepare_data(directory):
    features = []
    print(f"Starting data preparation in directory: {directory}")
    file_paths = [os.path.join(directory, f"{i}.mp3") for i in range(len(os.listdir(directory))) if os.path.isfile(os.path.join(directory, f"{i}.mp3"))]

    # Use ThreadPoolExecutor to parallelize file processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=24) as executor:
        results = executor.map(preprocess_audio, file_paths)

    # Filter out None results and convert to numpy array
    features = np.array([result for result in results if result is not None])
    print(f"Processed {len(features)} audio files out of {len(file_paths)}")

    return features


In [4]:
# Block 4: Preparing Data
train_features = prepare_data('train_mp3s/train_mp3s')
test_features = prepare_data('test_mp3s/test_mp3s')
train_labels = np.array(train_labels)

Starting data preparation in directory: train_mp3s/train_mp3s
Attempting to load file: train_mp3s/train_mp3s\0.mp3
Attempting to load file: train_mp3s/train_mp3s\1.mp3
Attempting to load file: train_mp3s/train_mp3s\2.mp3
Attempting to load file: train_mp3s/train_mp3s\3.mp3
Attempting to load file: train_mp3s/train_mp3s\4.mp3
Attempting to load file: train_mp3s/train_mp3s\5.mp3
Attempting to load file: train_mp3s/train_mp3s\6.mp3
Attempting to load file: train_mp3s/train_mp3s\7.mp3
Attempting to load file: train_mp3s/train_mp3s\8.mp3
Attempting to load file: train_mp3s/train_mp3s\9.mp3
Attempting to load file: train_mp3s/train_mp3s\10.mp3
Attempting to load file: train_mp3s/train_mp3s\11.mp3
Attempting to load file: train_mp3s/train_mp3s\12.mp3
Attempting to load file: train_mp3s/train_mp3s\13.mp3
Attempting to load file: train_mp3s/train_mp3s\14.mp3
Attempting to load file: train_mp3s/train_mp3s\15.mp3
Attempting to load file: train_mp3s/train_mp3s\16.mp3
Attempting to load file: train

In [ ]:
# Block 5: Model Training and Prediction
if len(train_features) > 0:
    train_data, val_data, train_labels, val_labels = train_test_split(
        train_features, train_labels, test_size=0.3, random_state=42)

    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=(train_features.shape[1], 1)),
        MaxPooling1D(2),
        Conv1D(128, kernel_size=3, activation='relu'),
        GlobalAveragePooling1D(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=100, batch_size=32)

    predictions = model.predict(test_features)
    predicted_labels = np.argmax(predictions, axis=1)
    submission = pd.DataFrame({'id': np.arange(len(predicted_labels)), 'category': predicted_labels})
    submission.to_csv('submission.csv', index=False)
else:
    print("No training data available.")

Epoch 1/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3226 - loss: 11.8343 - val_accuracy: 0.3718 - val_loss: 1.3362
Epoch 2/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.3688 - loss: 1.4985 - val_accuracy: 0.3707 - val_loss: 1.2949
Epoch 3/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.3786 - loss: 1.3126 - val_accuracy: 0.3814 - val_loss: 1.2460
Epoch 4/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.3970 - loss: 1.2619 - val_accuracy: 0.4436 - val_loss: 1.1832
Epoch 5/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 882us/step - accuracy: 0.4061 - loss: 1.2445 - val_accuracy: 0.5090 - val_loss: 1.1367
Epoch 6/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.4369 - loss: 1.2240 - val_accuracy: 0.5252 - val_loss: 1.1130
Epoch 7/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.4562 - loss: 1.1896 - val_accuracy: 0.5292 - val_loss: 1.0537
Epoch 8/1000
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.4831 - 